
# Azure Resource and Application Deployment Report

This notebook provides a detailed, step-by-step guide to creating Azure resources and deploying a containerized application using Azure services. Each step includes the necessary commands and explanations. 

---


## Step 1: Log in to Azure

In [ ]:

# Log in to your Azure account
az login



This command logs you into your Azure account. Make sure you're authenticated and select the appropriate subscription if prompted.

---


## Step 2: Create a Resource Group

In [ ]:

# Create a resource group in the East US region
az group create --name bouchaib_cli_gr --location eastus



A resource group is a logical container for managing related Azure resources. Replace `bouchaib_cli_gr` and `eastus` if you want a different name or region.

---


## Step 3: Create Azure Container Registry (ACR)

In [ ]:

# Create an Azure Container Registry
az acr create --resource-group bouchaib_cli_gr --name bouchaibacr --sku Basic --location eastus



Azure Container Registry (ACR) is used to store and manage your container images. Replace `bouchaibacr` with your preferred name.

---


## Step 4: Enable Admin Access for ACR

In [ ]:

# Enable admin access for ACR
az acr update -n bouchaibacr --admin-enabled true



This allows you to authenticate with ACR using admin credentials. Only enable it if needed.

---


## Step 5: Build Docker Image

In [ ]:

# Navigate to your project directory and build the Docker image
docker build -t malaria_app .



This builds your application image using the `Dockerfile` in the current directory. Replace `portfolio_project` with your preferred tag.

---


## Step 6: Tag and Push Docker Image to ACR

In [ ]:

# Tag the Docker image for ACR
docker tag portfolio_project bouchaibacr.azurecr.io/pmalaria_app:v1

# Log in to ACR
az acr login --name bouchaibacr

# Push the image to ACR
docker push bouchaibacr.azurecr.io/pomalaria_app:v1



- Replace `malaria_app` and `v1` with your image name and version.
- Verify the image is in ACR using:

```bash
az acr repository list --name bouchaibacr --output table
```

---


## Step 7: Deploy Container to Azure Container Instance (ACI)

In [ ]:

# Deploy container with public access
az container create `
  --resource-group bouchaib_cli_gr `
  --name portfolio-container `
  --image bouchaibacr.azurecr.io/pormalaria_app:v1 `
  --os-type Linux `
  --cpu 1 `
  --memory 1 `
  --ports 80 `
  --ip-address Public `
  --dns-name-label portfolio-app `
  --registry-login-server bouchaibacr.azurecr.io `
  --registry-username $(az acr credential show --name bouchaibacr --query "username" --output tsv) `
  --registry-password $(az acr credential show --name bouchaibacr --query "passwords[0].value" --output tsv)



This deploys the container to Azure Container Instances (ACI) with public access and assigns a DNS name (`portfolio-app`). Replace the DNS name label as needed.

---


## Step 8: Verify Deployment

In [ ]:

# Check container status and get its public FQDN
az container show --resource-group bouchaib_cli_gr --name malaria_app-container --query "{Status:instanceView.state,FQDN:ipAddress.fqdn}" --output table



- `Status` should be `Running`.
- Copy the FQDN and open it in your browser to test the app.

---


## Step 9: Clean Up Resources (Optional)

In [ ]:

# Delete the container instance
az container delete --resource-group bouchaib_cli_gr --name malaria_app-container --yes

# Delete the resource group (including all resources inside)
az group delete --name bouchaib_cli_gr --yes --no-wait



Run these commands if you no longer need the resources to avoid incurring unnecessary costs.

---


to get to know the dns nom de demaine

In [ ]:
az container show --resource-group bouchaib_cli_gr --name malaria_app-container --query "{FQDN:ipAddress.fqdn}" --output table
